<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/04_hsv-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-25 08:48:15--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12099 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.82K  --.-KB/s    in 0s      

2022-06-25 08:48:15 (86.4 MB/s) - ‘utils.py’ saved [12099/12099]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub


from utils import ModelCreator, get_train_val_ds, get_class_weight

In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
hsv_creator = ModelCreator(HUB_URL, 'HSV-model')
hsv_model = hsv_creator.make_model(img_size=IMG_SIZE)

Model: "HSV-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              12930622  
                                                                 
 dense (Dense)               (None, 16)                24592     
                                                                 
Total params: 12,955,214
Trainable params: 12,845,998
Non-trainable params: 109,216
_________________________________________________________________


In [10]:
callbacks = hsv_creator.get_callbacks()

In [11]:
history = hsv_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=15,
                        validation_data=val_ds,
                        class_weight=class_weight)

Epoch 1/15
519/519 [==============================] - 2021s 4s/step - loss: 18.3531 - auc: 0.6908 - accuracy: 0.6196 - val_loss: 0.9502 - val_auc: 0.8000 - val_accuracy: 0.7194
Epoch 2/15
519/519 [==============================] - 227s 435ms/step - loss: 11.4144 - auc: 0.8446 - accuracy: 0.7610 - val_loss: 0.9911 - val_auc: 0.7680 - val_accuracy: 0.6786
Epoch 3/15
519/519 [==============================] - 232s 444ms/step - loss: 9.0094 - auc: 0.8864 - accuracy: 0.8074 - val_loss: 0.9002 - val_auc: 0.8042 - val_accuracy: 0.7228
Epoch 4/15
519/519 [==============================] - 234s 448ms/step - loss: 6.8091 - auc: 0.9189 - accuracy: 0.8411 - val_loss: 0.8947 - val_auc: 0.8434 - val_accuracy: 0.7732
Epoch 5/15
519/519 [==============================] - 233s 446ms/step - loss: 5.1457 - auc: 0.9446 - accuracy: 0.8781 - val_loss: 0.8820 - val_auc: 0.8169 - val_accuracy: 0.7430
Epoch 6/15
519/519 [==============================] - 233s 447ms/step - loss: 3.9131 - auc: 0.9628 - accuracy:

In [14]:
history = hsv_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=20,
                          initial_epoch=9,
                          validation_data=val_ds,
                          class_weight=class_weight)

Epoch 10/20
519/519 [==============================] - 230s 437ms/step - loss: 1.0041 - auc: 0.9942 - accuracy: 0.9731 - val_loss: 1.2232 - val_auc: 0.8193 - val_accuracy: 0.8077
Epoch 11/20
519/519 [==============================] - 231s 443ms/step - loss: 1.5243 - auc: 0.9905 - accuracy: 0.9607 - val_loss: 1.2737 - val_auc: 0.8048 - val_accuracy: 0.7880
Epoch 12/20
519/519 [==============================] - 249s 478ms/step - loss: 0.7084 - auc: 0.9966 - accuracy: 0.9802 - val_loss: 1.3288 - val_auc: 0.8051 - val_accuracy: 0.7877
Epoch 13/20
519/519 [==============================] - 248s 476ms/step - loss: 0.4755 - auc: 0.9976 - accuracy: 0.9879 - val_loss: 1.4087 - val_auc: 0.8034 - val_accuracy: 0.7909
Epoch 14/20
519/519 [==============================] - 232s 445ms/step - loss: 0.6708 - auc: 0.9946 - accuracy: 0.9796 - val_loss: 1.4820 - val_auc: 0.7782 - val_accuracy: 0.7709
Epoch 15/20
519/519 [==============================] - 233s 447ms/step - loss: 0.3873 - auc: 0.9983 - acc

In [15]:
!cp ./logs/checkpoints/HSV-model.h5 ./drive/MyDrive/checkpoints/hsv-model-final.h5
